<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Изучаем-общую-информацию." data-toc-modified-id="1.-Изучаем-общую-информацию.-1">1. Изучаем общую информацию.</a></span></li><li><span><a href="#Вывод." data-toc-modified-id="Вывод.-2">Вывод.</a></span></li><li><span><a href="#3.-Предобработка-данных." data-toc-modified-id="3.-Предобработка-данных.-3">3. Предобработка данных.</a></span><ul class="toc-item"><li><span><a href="#3.1.-Обработка-пропусков." data-toc-modified-id="3.1.-Обработка-пропусков.-3.1">3.1. Обработка пропусков.</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.2">Вывод</a></span></li><li><span><a href="#3.2.-Замена-типа-данных." data-toc-modified-id="3.2.-Замена-типа-данных.-3.3">3.2. Замена типа данных.</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.4">Вывод</a></span></li><li><span><a href="#3.3.-Обработка-дубликатов." data-toc-modified-id="3.3.-Обработка-дубликатов.-3.5">3.3. Обработка дубликатов.</a></span></li><li><span><a href="#Вывод." data-toc-modified-id="Вывод.-3.6">Вывод.</a></span></li></ul></li><li><span><a href="#4.-Лемматизация." data-toc-modified-id="4.-Лемматизация.-4">4. Лемматизация.</a></span><ul class="toc-item"><li><span><a href="#Вывод." data-toc-modified-id="Вывод.-4.1">Вывод.</a></span></li></ul></li><li><span><a href="#5.-Категоризация-данных." data-toc-modified-id="5.-Категоризация-данных.-5">5. Категоризация данных.</a></span><ul class="toc-item"><li><span><a href="#Вывод." data-toc-modified-id="Вывод.-5.1">Вывод.</a></span></li></ul></li><li><span><a href="#6.-Исследование-данных." data-toc-modified-id="6.-Исследование-данных.-6">6. Исследование данных.</a></span></li><li><span><a href="#Общий-вывод." data-toc-modified-id="Общий-вывод.-7">Общий вывод.</a></span></li></ul></div>

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## 1. Изучаем общую информацию. 

In [1]:
import pandas as pd
from IPython.display import display
import numpy as np
from pymystem3 import Mystem
from collections import Counter

In [2]:
df = pd.read_csv('/datasets/data.csv')
df.sample(9)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12182,0,-5248.890243,44,среднее,1,Не женат / не замужем,4,F,сотрудник,1,216240.576190,автомобили
21199,0,-207.551107,36,высшее,0,женат / замужем,0,F,компаньон,0,171501.359551,приобретение автомобиля
11578,0,370587.572555,52,среднее,1,женат / замужем,0,F,пенсионер,0,147982.225473,строительство недвижимости
532,0,-1935.511889,51,среднее,1,гражданский брак,1,M,компаньон,0,102787.514906,сыграть свадьбу
20134,0,-622.426583,51,среднее,1,женат / замужем,0,M,сотрудник,0,251871.135539,сделка с подержанным автомобилем
2154,1,-2854.301161,46,среднее,1,женат / замужем,0,M,сотрудник,0,124175.807568,сделка с автомобилем
18723,1,-695.274612,30,Высшее,0,женат / замужем,0,F,компаньон,0,128030.151557,заняться образованием
892,0,350026.220328,42,среднее,1,женат / замужем,0,M,пенсионер,0,190005.440576,операции с недвижимостью
11786,0,-1165.197276,47,среднее,1,гражданский брак,1,F,сотрудник,0,64592.505706,сыграть свадьбу


Читаем файл `data.csv` и сохраняем его в переменной __data__.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Смотрим общую информацию о таблице __df__.

Всего у нас 12 столбцов с различными типами данных: int64, float64 и object.

Разберём столбцы и что в них содержится:

	• children — количество детей в семье   
	• days_employed — общий трудовой стаж в днях   
	• dob_years — возраст клиента в годах  
	• education — уровень образования клиента  
	• education_id — идентификатор уровня образования   
	• family_status — семейное положение   
	• family_status_id — идентификатор семейного положения  
	• gender — пол клиента   
	• income_type — тип занятости   
	• debt — имел ли задолженность по возврату кредитов   
	• total_income — ежемесячный доход   
	• purpose — цель получения кредита  

In [4]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


## Вывод.

Данные не идеальны и сперва их нужно обработать:
- в столбцах __days_employed__ и __total_income__ лучше перевести в целые числа, ещё в них есть пропуски.
- в столбце __children__ есть отрицательные значения, что является странным. Но и максимальное значение - 20 детей, хотя такое возможно.
- в __days_employed__ тоже есть отрицательные значения.
- в __dob_years__ возраст клиента равняется 0.
- __education__ нужно привести к общему регистру, лучше к нижнему.

## 3. Предобработка данных.

### 3.1. Обработка пропусков.

In [5]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Итак, у нас по 2174 пропущенных значений в двух стобцах. 
- __days_employed__ - возможно не заполнен, т.к. у кого-то нет стажа, либо это ошибка, то что есть отрицательные значения это явно какая-то ошибка - переведу эти значения в положительные. 
- __total_income__ - тоже может быть не заполнен по причине ошибки либо клиент сам не заполнил эти данные. Что конечно странно, это всё-таки кредиты.

Переведём __days_employed__ из отрицаленых в положительные значения и посмотрим на медиану.

In [6]:
df['days_employed'] = df['days_employed'].abs()
print('Средний трудовой стаж', df['total_income'].mean(),'дней.')
print('Медиана по трудовому стажу',df['days_employed'].median(),'дней.')

Средний трудовой стаж 167422.30220817294 дней.
Медиана по трудовому стажу 2194.220566878695 дней.


In [7]:
days_employed_median = df['days_employed'].median()
df['days_employed'] = df['days_employed'].fillna(days_employed_median)

С __days_employed__ разобрались, пропуски заполнили по медиане, т.к. среднее 167422 дней (458 лет) - есть выбросы в данных.

In [8]:
print('Средний ежемесячный доход', df['total_income'].mean())
print('Медианный ежемесячный доход', df['total_income'].median())

Средний ежемесячный доход 167422.30220817294
Медианный ежемесячный доход 145017.93753253992


In [9]:
total_income_mean = df['total_income'].mean()
df['total_income'] = df['total_income'].fillna(total_income_mean)

__total_income__ лучше заполнить по среднему т.к. нет явных выбросов, которые сильно исказили бы данные.

In [10]:
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

В столбце __dob_years__ есть значение равное 0. Сомнительно что человек обратившийся за кредитом может быть младенцем. Это нужно исправить. Посмотрим сколько таких значений и сколько тех кому меньше 19 лет, т.к. человек моложе не сможет оформлять кредит:

Значит все значения тех кому меньше 19 лет равно 0. Заменим их на среднее:

In [11]:
dob_years_mean = int(df['dob_years'].mean())
df['dob_years'] = df['dob_years'].replace({0 : dob_years_mean})

Проверим наши изменения:

In [12]:
print('Строк "dob_years" равных 0 - ', df[df['dob_years'] == 0].count()[0])

Строк "dob_years" равных 0 -  0


Посмотрим на данные столбца __children__:

In [13]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Переведём отрицательные значения в положительные и __20__ в __2__. Скорее всего это технические ошибки.

In [14]:
df['children'] = df['children'].abs()
df['children'] = df['children'].replace({20 : 2})

Проверим столбец __children__:

In [15]:
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Вывод

Отлично, пропущенных и кривых значений больше нет.

### 3.2. Замена типа данных.

Переведём столбцы __days_employed__ и __total_income__ в целые числа методом `astype`.

In [16]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

Проверим результат, для этого вызовем метод `info`.

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Перевели данные в целые числа, с ними проще работать.

### 3.3. Обработка дубликатов.

Для того чтобы избавиться от дубликатов нужно столбец __education__ привести к общему регистру.

In [18]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Переведём информацию об уровне образования клиентов в нижний регистр, сохраним и проверим результат:

In [19]:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Теперь проверим таблицу на дубликаты.

In [20]:
df.duplicated().sum()

71

Колличество дубликатов - 71. Удаляем. Проверяем ещё раз на их отсутствие:

In [21]:
df = df.drop_duplicates().reset_index(drop = True)
df.duplicated().sum()

0

### Вывод.

От дубликатов избавились.

## 4. Лемматизация.

Поместим значения из из стобца __purpose__ в переменную __list_purpose__. Оставим уникальные значения и уберём пропуски.

In [22]:
list_purpose = df['purpose'].drop_duplicates().reset_index(drop= True).to_frame()
list_purpose = list_purpose.fillna('')

Проведём лемматизацию уникальных значений:

In [23]:
m = Mystem()
list_purpose['purpose_lem'] = list_purpose['purpose'].apply(m.lemmatize)

Посчитаем число их уникальных значений:

In [24]:
print(Counter(list_purpose['purpose_lem'].sum()))

Counter({' ': 59, '\n': 38, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


Добавим леммы в __DataFrame__.

In [25]:
df = pd.merge(left = df, right = list_purpose, on = 'purpose', how = 'left' )

Из списка возьмём самые популярные значения и запишем их в __unique_purpose__.

In [26]:
unique_purpose = ['автомобиль', 'жилье', 'свадьба', 'образование', 'недвижимость']

Определим функцию __purpose_func__ с аргументом слов из ранее созданного списка для проведения лемматизации и разбивки списка по
категориям:

In [27]:
def purpose_func(words):
 
    for word in unique_purpose:
        if word in words:
            return word
    return 'другое'

Сохраним результат в новый столбец __purpose_category__ и заменим ___жильё___ на ___недвижимость___.
Посомтрим на таблицу и проверим результат.

In [28]:
df['purpose_category'] = df['purpose_lem'].apply(purpose_func)
df['purpose_category'] = df['purpose_category'].replace({'жилье' : 'недвижимость'})
df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lem,purpose_category
17608,0,2735,46,среднее,1,женат / замужем,0,M,сотрудник,0,145521,высшее образование,"[высокий, , образование, \n]",образование
3816,1,5259,35,высшее,0,женат / замужем,0,F,сотрудник,0,140123,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
11734,0,316,24,среднее,1,Не женат / не замужем,4,M,сотрудник,0,138426,образование,"[образование, \n]",образование
3706,1,2194,29,среднее,1,в разводе,3,F,сотрудник,0,167422,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость
9559,0,2005,50,среднее,1,гражданский брак,1,F,сотрудник,0,106993,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба


In [29]:
df['purpose_category'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

### Вывод.

Датафрейм готов к категорицации.

## 5. Категоризация данных.

Чтобы сделать выводы о платежеспособности клиентов нам нужны следующие категории:
- семейное положение;
- по количеству детей;
- по уровню дохода;
- цель займа.

Проверим категорию семейного положения __family_status__ с __family_status_id__:

In [30]:
df.groupby(['family_status', 'family_status_id']).count()

,,children,days_employed,dob_years,education,education_id,gender,income_type,debt,total_income,purpose,purpose_lem,purpose_category
family_status,family_status_id,,,,,,,,,,,,
Не женат / не замужем,4,2810,2810,2810,2810,2810,2810,2810,2810,2810,2810,2810,2810
в разводе,3,1195,1195,1195,1195,1195,1195,1195,1195,1195,1195,1195,1195
вдовец / вдова,2,959,959,959,959,959,959,959,959,959,959,959,959
гражданский брак,1,4151,4151,4151,4151,4151,4151,4151,4151,4151,4151,4151,4151
женат / замужем,0,12339,12339,12339,12339,12339,12339,12339,12339,12339,12339,12339,12339


Всё хорошо, пары уникальны.

Создадим функцию, которая оценивает количество детей. Присвоим следующие категории:
- детей нет - 0;
- есть дети - 1-2;
- многодетные - 3 и более.

In [31]:
def children_cat(row):
    if row == 0:
        return 'нет детей'
    if row > 0 and row <= 2:
        return 'есть дети'
    if row > 2:
        return 'многодетные'

Добавим столбец __children_cat__ в DataFrame.

In [32]:
df['children_cat'] = df['children'].apply(children_cat)
df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lem,purpose_category,children_cat
8611,2,527,30,среднее,1,женат / замужем,0,F,компаньон,0,177986,покупка жилья,"[покупка, , жилье, \n]",недвижимость,есть дети
5114,0,1411,54,среднее,1,Не женат / не замужем,4,F,компаньон,0,162401,жилье,"[жилье, \n]",недвижимость,нет детей
4053,0,376698,58,высшее,0,женат / замужем,0,M,пенсионер,0,89527,заняться образованием,"[заниматься, , образование, \n]",образование,нет детей
8768,0,394844,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,149613,высшее образование,"[высокий, , образование, \n]",образование,нет детей
19699,2,349,30,начальное,3,женат / замужем,0,M,сотрудник,0,111215,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]",автомобиль,есть дети


Добавим столбец с категоризацией уровня дохода, воспользуемся методом `quantile`, и укажем следующие категории:
- до 25% - _низкий_
- от 25 до 50% - _средний_
- от 50 до 75% - _высокий_
- более 75% - _очень высокий_.

In [33]:
df['income_cat'] = pd.qcut(df['total_income'],4, ['низкий', 'средний', 'высокий', 'очень высокий'])
df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lem,purpose_category,children_cat,income_cat
17494,1,620,31,среднее,1,Не женат / не замужем,4,M,сотрудник,0,258218,заняться образованием,"[заниматься, , образование, \n]",образование,есть дети,очень высокий
13228,0,343175,58,среднее,1,вдовец / вдова,2,F,пенсионер,0,199594,автомобили,"[автомобиль, \n]",автомобиль,нет детей,очень высокий
10102,0,3041,42,среднее,1,женат / замужем,0,M,сотрудник,0,212025,строительство жилой недвижимости,"[строительство, , жилой, , недвижимость, \n]",недвижимость,нет детей,очень высокий
6517,0,341341,41,среднее,1,женат / замужем,0,M,пенсионер,0,252057,недвижимость,"[недвижимость, \n]",недвижимость,нет детей,очень высокий
11365,1,73,38,высшее,0,гражданский брак,1,F,сотрудник,0,116496,покупка своего жилья,"[покупка, , свой, , жилье, \n]",недвижимость,есть дети,средний


### Вывод.

Теперь можно проанализировать платёжеспособность клиентов на основании созданных категорий и задолжности по возврату кредитов.

## 6. Исследование данных.

**Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [34]:
df.groupby('children_cat')['debt'].agg(['count','sum','mean']).sort_values(by = 'mean')

,count,sum,mean
children_cat,,,
нет детей,14091,1063,0.075438
многодетные,380,31,0.081579
есть дети,6983,647,0.092654


Заёмщики без детей имеют наименьший процент по просрочке.

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [35]:
df.groupby('family_status')['debt'].agg(['count','sum','mean']).sort_values(by = 'mean')

,count,sum,mean
family_status,,,
вдовец / вдова,959,63,0.065693
в разводе,1195,85,0.071130
женат / замужем,12339,931,0.075452
гражданский брак,4151,388,0.093471
Не женат / не замужем,2810,274,0.097509


Заёмщики которые никогда не состояли в оффициальном браке более склонны к просрочке кредита.

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [36]:
df.groupby('income_cat')['debt'].agg(['count','sum','mean']).sort_values(by = 'mean')

,count,sum,mean
income_cat,,,
очень высокий,5364,383,0.071402
низкий,5364,427,0.079605
высокий,5363,463,0.086332
средний,5363,468,0.087265


А вот это уже интересно, оказалось что заёмщики с __очень высоким__, либо __низким__ доходом наиболее надёжны.

**Как разные цели кредита влияют на его возврат в срок?**

In [37]:
df.groupby('purpose_category')['debt'].agg(['count','sum','mean']).sort_values(by = 'mean')

,count,sum,mean
purpose_category,,,
недвижимость,10811,782,0.072334
свадьба,2324,186,0.080034
образование,4013,370,0.092200
автомобиль,4306,403,0.093590


Заёмщики, которые берут кредит на жильё наиболее ответсвенны. Да и по колличеству их больше всего.

**Влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок?**

Cделаем сводную таблицу:

In [38]:
df.pivot_table(index='family_status',columns='children_cat',values='debt',aggfunc=['count','sum','mean'])

count                             sum              \
children_cat          есть дети многодетные нет детей есть дети многодетные   
family_status                                                                 
Не женат / не замужем       538          10      2262        62           2   
в разводе                   399          12       784        29           1   
вдовец / вдова              105           7       847        10           0   
гражданский брак           1355          66      2730       151           8   
женат / замужем            4586         285      7468       395          20   

                                     mean                        
children_cat          нет детей есть дети многодетные нет детей  
family_status                                                    
Не женат / не замужем       210  0.115242    0.200000  0.092838  
в разводе                    55  0.072682    0.083333  0.070153  
вдовец / вдова               53  0.095238    0.000000  0.062574  
гражданский брак            229  0.111439    0.121212  0.083883  
женат / замужем             516  0.086132    0.070175  0.069095

Тут появляются исключения по сравнению с прошлым выводом: в категории __вдовец/вдова__ при наличии детей значительно вырастает процент просрочки.

## Общий вывод.

Семейное положение влияет на платежеспособность клиента, но ещё сильнее влияет колличество детей.